<a href="https://colab.research.google.com/github/Robert-099/bertclassificacao/blob/main/classificaca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classificação de Comentários com BERT

Treinando modelo para as classes: **onça, fake_news, notícia**

In [2]:
import pandas as pd
url = "https://docs.google.com/spreadsheets/d/17aHYyRNfbmde8bVOR_HX_BmNUEdkygPuaGO4lJj26jg/export?format=csv"
df = pd.read_csv(url)
df = df[['comment_text','onca','fake news','notícia']]
df.head()

,comment_text,onca,fake news,notícia
0,Solta no congresso,neutro,não,neutra
1,"vídeo direto, respeitoso, sem sensacionalismo ...",neutro,não,boa
2,Essa reportagem foi a melhor parabéns,neutro,não,boa
3,As patas 🐾 das onças no chão foi um aviso pra ...,neutro,não,neutra
4,Se o cara viu que onça estava rondando a casa ...,neutro,não,neutra


In [4]:
def choose_label(row):
    if row['onca'] != 'neutro': return 'onca'
    if row['fake news'] != 'não': return 'fake_news'
    if row['notícia'] != 'neutra': return 'noticia'
    return None

df['label'] = df.apply(choose_label, axis=1)
df = df.dropna(subset=['label'])
df['label_id'] = df['label'].map({'onca':0,'fake_news':1,'noticia':2})
df.head()

/tmp/ipython-input-1992462258.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label_id'] = df['label'].map({'onca':0,'fake_news':1,'noticia':2})


,comment_text,onca,fake news,notícia,label,label_id
1,"vídeo direto, respeitoso, sem sensacionalismo ...",neutro,não,boa,noticia,2
2,Essa reportagem foi a melhor parabéns,neutro,não,boa,noticia,2
6,Equipe de reportagem está de parabéns! Minha s...,neutro,não,boa,noticia,2
7,Parabéns pelo vídeo. Sem sensacionalismo e inf...,neutro,não,boa,noticia,2
9,"É estratégia da onça, observar e marcar o terr...",negativo,não,neutra,onca,0


In [5]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')
MAX_LEN = 128

def encode(text):
    return tokenizer.encode_plus(text, truncation=True, padding='max_length', max_length=MAX_LEN, return_tensors='pt')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/647 [00:00<?, ?B/s]

In [6]:
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

class NewsDataset(Dataset):
    def __init__(self, df):
        self.texts = df['comment_text'].tolist()
        self.labels = df['label_id'].tolist()
    def __len__(self): return len(self.texts)
    def __getitem__(self, idx):
        e = encode(self.texts[idx])
        return {
            'input_ids': e['input_ids'].squeeze(),
            'attention_mask': e['attention_mask'].squeeze(),
            'labels': torch.tensor(self.labels[idx])
        }

train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['label_id'])
train_loader = DataLoader(NewsDataset(train_df), batch_size=16, shuffle=True)
val_loader   = DataLoader(NewsDataset(val_df), batch_size=16)

In [8]:
from transformers import BertForSequenceClassification
from torch.optim import AdamW

model = BertForSequenceClassification.from_pretrained('neuralmind/bert-base-portuguese-cased', num_labels=3)
optimizer = AdamW(model.parameters(), lr=2e-5)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(29794, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [2]:
from tqdm import tqdm
epochs = 3

for epoch in range(epochs):
    print(f'Epoch {epoch+1}/{epochs}')
    model.train()
    total = 0
    for batch in tqdm(train_loader):
        optimizer.zero_grad()
        ids = batch['input_ids'].to(device)
        mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        out = model(ids, attention_mask=mask, labels=labels)
        loss = out.loss
        loss.backward()
        optimizer.step()
        total += loss.item()
    print('Train loss:', total/len(train_loader))

Epoch 1/3


NameError: name 'model' is not defined

In [ ]:
model.eval()
correct = total = 0
with torch.no_grad():
    for batch in val_loader:
        ids = batch['input_ids'].to(device)
        mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        out = model(ids, attention_mask=mask)
        preds = out.logits.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
print('Accuracy:', correct/total)